# 色空間を使った画像処理

このノートブックでは、色空間を使った画像処理について学びます。

## 目次
1. [肌色検出](#肌色検出)
2. [背景除去](#背景除去)
3. [色補正](#色補正)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["axes.unicode_minus"] = False
np.random.seed(42)
print("ライブラリのインポートが完了しました。")

## 肌色検出

In [ ]:
# HSV色空間を使った肌色検出
# 肌色はHSV色空間で検出しやすい

# サンプル画像の作成（肌色を含む）
img = np.zeros((200, 200, 3), dtype=np.uint8)
# 肌色の領域を作成
img[50:150, 50:150] = [200, 150, 120]  # 肌色に近い色

# RGBからHSVに変換
img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

# 肌色の範囲を定義（HSV）
# H: 0-20（赤からオレンジ）
# S: 50-255（ある程度の彩度）
# V: 50-255（ある程度の明度）
lower_skin = np.array([0, 50, 50])
upper_skin = np.array([20, 255, 255])

# 肌色のマスクを作成
skin_mask = cv2.inRange(img_hsv, lower_skin, upper_skin)

# マスクを適用
skin_detected = cv2.bitwise_and(img, img, mask=skin_mask)

# 結果の表示
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(img)
axes[0, 0].set_title('元の画像')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_hsv)
axes[0, 1].set_title('HSV色空間')
axes[0, 1].axis('off')

axes[1, 0].imshow(skin_mask, cmap='gray')
axes[1, 0].set_title('肌色マスク')
axes[1, 0].axis('off')

axes[1, 1].imshow(skin_detected)
axes[1, 1].set_title('検出された肌色')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("HSV色空間での肌色検出:")
print("- H（色相）: 0-20度（赤からオレンジ）")
print("- S（彩度）: 50-255（ある程度の彩度が必要）")
print("- V（明度）: 50-255（暗すぎない）")
print("- cv2.inRange()で範囲内のピクセルを検出")

## 背景除去

In [ ]:
# HSV色空間を使った背景除去（色に基づく）
# 特定の色の背景を除去します

# サンプル画像の作成（緑色の背景）
img = np.zeros((200, 200, 3), dtype=np.uint8)
img[:, :] = [0, 200, 0]  # 緑色の背景
img[50:150, 50:150] = [200, 100, 50]  # オブジェクト（肌色）

# RGBからHSVに変換
img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

# 緑色の範囲を定義
lower_green = np.array([40, 50, 50])
upper_green = np.array([80, 255, 255])

# 緑色のマスクを作成
green_mask = cv2.inRange(img_hsv, lower_green, upper_green)

# 背景を除去（緑色以外を残す）
background_removed = img.copy()
background_removed[green_mask > 0] = [255, 255, 255]  # 背景を白色に

# または、透明な背景にする（アルファチャンネル追加）
img_rgba = cv2.cvtColor(img, cv2.COLOR_RGB2RGBA)
img_rgba[:, :, 3] = 255  # アルファチャンネルを追加
img_rgba[green_mask > 0, 3] = 0  # 緑色の部分を透明に

# 結果の表示
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(img)
axes[0, 0].set_title('元の画像（緑色背景）')
axes[0, 0].axis('off')

axes[0, 1].imshow(green_mask, cmap='gray')
axes[0, 1].set_title('緑色マスク')
axes[0, 1].axis('off')

axes[1, 0].imshow(background_removed)
axes[1, 0].set_title('背景除去（白色）')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_rgba)
axes[1, 1].set_title('背景除去（透明）')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("HSV色空間での背景除去:")
print("- 特定の色の範囲を検出")
print("- マスクを作成して背景を除去")
print("- チャロマキー（色に基づく背景除去）の原理")

## 色補正

In [ ]:
# LAB色空間を使った色補正
# LAB色空間は知覚的に均一なので、色補正に適しています

# サンプル画像の作成
img = np.random.randint(0, 256, (200, 200, 3), dtype=np.uint8)
img[50:150, 50:150] = [150, 100, 80]  # 特定の色の領域

# RGBからLABに変換
img_lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)

# L（明度）チャンネルを調整
img_lab_adjusted = img_lab.copy()
img_lab_adjusted[:, :, 0] = np.clip(img_lab[:, :, 0] * 1.2, 0, 100)  # 明度を20%増加

# A（緑-赤）チャンネルを調整
img_lab_adjusted[:, :, 1] = np.clip(img_lab[:, :, 1] + 10, -128, 127)  # 赤みを増加

# B（青-黄）チャンネルを調整
img_lab_adjusted[:, :, 2] = np.clip(img_lab[:, :, 2] - 10, -128, 127)  # 黄色みを減少

# LABからRGBに戻す
img_adjusted = cv2.cvtColor(img_lab_adjusted.astype(np.uint8), cv2.COLOR_LAB2RGB)

# 結果の表示
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(img)
axes[0, 0].set_title('元の画像')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_adjusted)
axes[0, 1].set_title('色補正後')
axes[0, 1].axis('off')

# LABチャンネルの比較
axes[1, 0].imshow(img_lab[:, :, 0], cmap='gray')
axes[1, 0].set_title('元のLチャンネル')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_lab_adjusted[:, :, 0], cmap='gray')
axes[1, 1].set_title('調整後のLチャンネル')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("LAB色空間での色補正:")
print("- L（明度）: 0-100")
print("- A（緑-赤）: -128から+127")
print("- B（青-黄）: -128から+127")
print("- 知覚的に均一なので、色補正に適している")